# US Immigration Data
### Data Engineering Capstone Project

#### Project Summary
I will use the US Immigration dataset I94 to analyse information about German residents travelling to the USA.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# I want to suppress warning messages
import warnings
warnings.filterwarnings('ignore')

### Step 1: Scope the Project and Gather Data

#### Scope 
I'm interested in analysing the behavior of German residents travelling to the USA. I want this project to answer the below questions:

* Which types of Visas do Germans use?
* what is their travelling patterns over the year?
* what are their preferred states to reside in during their visits to USA?

To answer these questions, I will use the below datasets:

1. US immigration I94 dataset for all months during 2016
2. An external JSON file with the countries ISO codes to be able to extract German residents only from the Immigration dataset.
3. The US demographic dataset to extract the US states full name, as the states included in immigration datasets are abbreviated.

I will be using Pandas and matplotlib libraries for my analysis


In [3]:
# Let's start by takeing a look at the sample file to understand the components of Immigration dataset
df = pd.read_csv('immigration_data_sample.csv', index_col=0)

In [4]:
df.head()

,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [5]:
df.columns

Index(['cicid', 'i94yr', 'i94mon', 'i94cit', 'i94res', 'i94port', 'arrdate',
       'i94mode', 'i94addr', 'depdate', 'i94bir', 'i94visa', 'count',
       'dtadfile', 'visapost', 'occup', 'entdepa', 'entdepd', 'entdepu',
       'matflag', 'biryear', 'dtaddto', 'gender', 'insnum', 'airline',
       'admnum', 'fltno', 'visatype'],
      dtype='object')

In [6]:
len(df.columns)

28

Using Chatgpt, here are the description of the columns,

* CICID: A unique identifier assigned to each individual record in the dataset.
* i94yr: The 4-digit year of the visitor's arrival in the United States.
* i94mon: The numeric month of the visitor's arrival in the United States.
* i94cit: The visitor's country of citizenship.
* i94res: The visitor's country of residence.
* i94port: The port of entry where the visitor arrived in the United States.
* arrdate: The arrival date of the visitor in the United States.
* i94mode: The mode of transportation used by the visitor to enter the United States.
* i94addr: The state where the visitor intends to reside in the United States.
* depdate: The departure date of the visitor from the United States.
* i94bir: The visitor's age at the time of arrival in the United States.
* i94visa: The type of visa the visitor used to enter theUnited States, such as business, pleasure, or student.
* count: The number of people included in the record.
* dtadfile: The date on which the data was added to the I94 file.
* visapost: The US embassy or consulate where the visitor obtained their visa.
* occup: The visitor's occupation.
* entdepa: The arrival flag - indicates the reason for the visitor's arrival in the US.
* entdepd: The departure flag - indicates the reason for the visitor's departure from the US.
* entdepu: Update flag - indicates if the arrival record was updated, typically for a change of address or other similar reasons.
* matflag: Match flag - indicates if the arrival and departure records match.
* biryear: The visitor's birth year.
* dtaddto: The date until which the visitor is admitted to stay in the US.
* gender: The visitor's gender.
* insnum: "Immigration and Naturalization Service (INS) number," which was a unique identifier assigned by the US government to individuals who applied for immigration benefits or who were admitted to the US as non-immigrant visitors.
* airline: The airline used by the visitor to travel to the US.
* admnum: Admission number - a unique number assigned to each arrival record.
* fltno: The flight number used by the non-immigrant visitor to enter the United States.
* visatype: The type of visa that a non-immigrant visitor used to enter the United States.

### Step 2: Explore and Assess the Data

I can see that column i94res includes a numerical value which represents the visitor's country of residence. I'm interested in retrieving the names of these countries. So, I uploaded a Json file that contains the ISO numerical code and the corresponding country name. Now let's use it.

In [7]:
codes=pd.read_json('countries.json')
codes.head(1)

,alpha2,alpha3,id,name
0,af,afg,4,Afghanistan


I'm Interested in analysing the data of German residents, so I will get its ISO code from the Codes dataset and use it to filter the data in the immigration dataset.

In [8]:
grmn_cd=codes[codes['name']=='Germany']['id'].iloc[0]
grmn_cd

276

Now let's get the immigration data for each month, and filter only the german residents records. I'm interested in exploring the types of visas column (visatype) and the state in which visitors will reside (i94addr), I will also keep the month column to be used in the analysis. 

I will ignore the count of persons within the same record that exists in column "count", and assume each record represents 1 person.

For each separate month, I will do 3 steps:

1. Filter and keep German residents
2. Delete the duplicates of the unique identifier 'cicid'
3. Delete the null values, and if their size is large (more than an assumed threshold), a warning message will display.

In [9]:
# set null values % threshold
thrshold= 0.3

def monthly_data(month):
    """Takes in a month dataset and returns a processed dataset with german residents"""
    
    data=pd.read_sas('../../data/18-83510-I94-Data-2016/i94_{}16_sub.sas7bdat'.format(month), 'sas7bdat', encoding="ISO-8859-1")
    data=data[['cicid','i94mon','i94res','i94addr','visatype']]
    grmn=data[data['i94res']==grmn_cd].drop_duplicates(subset='cicid')
    rto=(len(grmn)-grmn.count().min())/len(grmn)
    [print("****{} WARNING: Number of Missing Values Exceeds Threshold****".format(month.upper())) if rto>thrshold else None][0]
    grmn.dropna(inplace=True)
    return grmn

In [10]:
# prepare a list with all months
months=['jan', 'feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec']

As a quality check, I will run the monthly_data function on all months and display the first row only just to make sure files were read normally and no month has a high number of missing data

In [ ]:
for month in months:
    print('-----------------{}-----------------'.format(month.upper()))
    print(monthly_data(month).head(1))
    print()

-----------------JAN-----------------
       cicid  i94mon  i94res i94addr visatype
3136  5028.0     1.0   276.0      NY       F1

-----------------FEB-----------------
       cicid  i94mon  i94res i94addr visatype
1800  2656.0     2.0   276.0      NV       WT

-----------------MAR-----------------


Next, We will concatenate all months into one dataset, and drop the "i94res" column since it only include German residents' code

In [ ]:
german=pd.concat([monthly_data(month) for month in months], ignore_index=True)
german.drop(labels='i94res', inplace=True, axis=1)

In [ ]:
german.head(2)

In [ ]:
# Let's ensure that the # rows are more than 1 million (project requirement)
len(german)

In [ ]:
# Now let's read the U.S. City Demographic Data

df_demo=pd.read_csv('us-cities-demographics.csv',delimiter=';')
df_demo.head(1)

In [ ]:
# I'm only interested in the columns "State Code" & "State". I want to use them to form a dict

st=df_demo[['State Code','State']].set_index('State Code','State').to_dict()
st_dict=st['State']
print(len(st_dict))
st_dict

Now we have the data of 49 US state names and their codes. let's check the codes that exist in the immigration dataset

In [ ]:
imm_cd=german['i94addr'].value_counts()

imc=imm_cd.to_dict()
imc

In [ ]:
len(imc)

There seems to be alot of codes that exist in the immigration dataset while not in the demographic dataset. Let's see the codes of the missing states

In [ ]:
no=[(k, imc[k])for k in imc.keys()if k not in st_dict.keys()]
no

The number of records with unknown state name seems to be high, specially for the first 2 states ('GU' & 'MP'). We will not be able to drop them since they represent around 25% of the data. Since we are building a pipeline, then it's important to avoid hard coding steps by avoiding manual search for these two values. The solution will be to use a new dataset that contains the state name and the corresponding state code

In [ ]:
full_code=pd.read_csv('georef-united-states-of-america-zc-point@public.csv',delimiter=';')
full_code.head(1)

In [ ]:
# Let's get the required dictionary from our new dataset

stat=full_code[['Official USPS State Code','Official State Name']].set_index('Official USPS State Code','Official State Name').to_dict()
stat_dict=stat['Official State Name']
len(stat_dict)

In [ ]:
# Once again, let's see if there is a missing state code
non=[(k, imc[k])for k in imc.keys()if k not in stat_dict.keys()]
non

This time, the number of states with unknown code is smaller. I will replace these states' names with 'Others'

In [ ]:
stat_dict.update({key: 'Others' for key in imc.keys() if key not in stat_dict})

In [ ]:
print(len(stat_dict))
stat_dict

In [ ]:
# retrieve the full state name and add it to the german dataset
german['State']= german['i94addr'].apply(lambda x: stat_dict[x])
german.head(2)

In [ ]:
# I want to display a warning if the number of 'Others' state exceeded a certain threshold, 
# this will be helpful if the file was updated to ensure that the # of others will not be large in the new records.

st_thrsh=0.15
others_ratio=german[german['State']=='Others']['State'].count()/len(german)

[print("***** WARNING: Number of unknown (Others) states Exceeds Threshold****") if others_ratio>st_thrsh else None][0]

By Now, our dataset has reached its final form and is now ready for our analysis

Now let's answer our questions:
Which types of Visas do Germans use?

In [ ]:
visas=german['visatype'].value_counts()
visas

In [ ]:
plt.bar(list(visas.index), list(visas))
plt.title('Types of Visas used by German residents during their visit to USA')
plt.xlabel('Visa Type')
plt.ylabel('Visa Count')
plt.xticks(rotation=90)
#ax.bar_label(bars)
plt.box(False)
#plt.gca().invert_yaxis();

It seems that "WT" & "GMT" are the most populat visa types. Now let's check their travelling pattern over the year.

In [ ]:
pat=german['i94mon'].value_counts()
pat.sort_index(inplace=True)
pat

In [ ]:
plt.figure(figsize=(6,3))
plt.bar(x=pat.index, height=pat)
plt.title("Number of German Residents' Visits to USA per Month")
plt.xlabel('Month')
plt.ylabel('count');
plt.xticks(pat.index);

The count over month is a bit similar with no spikes (except for a decrease in March & April)

what are their preferred states to reside in during their visits to USA? I will display the top 10

In [ ]:
pref=german['State'].value_counts()
pref[:10].index

In [ ]:
plt.bar(list(pref[:10].index), list(pref[:10]))

plt.title('Top 10 Preferred States' +  "\n" + 'by German Residents', loc='right')
plt.xlabel('States')
plt.xticks(rotation=90)
plt.ylabel('Count')

plt.box(False);

It seems that Guam & California are the go-to states for German residents.

Now we have answers for our questions!

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
The data model used in this project is Star Schema. The reason for choosing it is that the datasets used are one fact table (Immigration dataset), and 2 dimensions tables (country ISO codes & states abbrev. table) which were used to complete the data in the main Immigration dataset and make it reach to a form that will be ready for analysis.

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Retrieve the immigration data from its source
2. Create a function (monthly_data) that takes in raw data files, does the required pre-processing, and displays the needed warnings-if any-.
3. Use dimention table (ISO code) to filter the required data (German residents) 
4. Join the fact table with dimentions table to retrieve the required data (State full name), and display a warning if the number of unknown states increase, so that in future runs after the files update, we could ensure that we don't have a huge number of missing state name. We also used "Others" to fill missing/incorrect state name.

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model

Build the data pipelines to create the data model: Already done in the above steps

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks:

Check #1: Checking the % of null values before dropping them, and displaying an error message if the % exceeded a given threshold.

Check #2: Checking the % of states with unknow names (refereed to as "others") and displaying a warning message if the % exceeded a given threshold.

Check #3: Dropping duplicated rows using the unique identifier column "cicid"

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file (Included in a separate file).

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.

I used only Jupyter notebook for the project, since the data size was still small and all project stages could be successfully run through the notebook.

* Propose how often the data should be updated and why.

From the analytics point of view, this data should be run once per month, since the analysis was built on month granularity. So any change in data behavior should be detected on the month level not on weekly or daily...etc specially that the dataset doesn't include columns for weeks or days timestamp. The smallest time level included is the month column. 

* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 
 I would upload the datasets to AWS and use it for analysis, since the data size will increase and it won't be efficient to run it in the notebook

 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 
I would use airflow and create a DAG with a scheduled daily run

 * The database needed to be accessed by 100+ people.
 
 I would upload all datasets on the cloud and use its storage capacity, and create different roles with different access for each group to be able to access the data with their proposed limitation.